In [1]:
######################
## import libraries ##
######################
import numpy as np
np.random.seed(1081565)
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [2]:
def simulate_dataset(n=2000000, A_on_Y=0):

    ## Z: “randomized” binary intervention assignment (0, 1)    
    ## A: observed binary intervention (0, 1)
    ## L: direct binary determinant of intervention A (0, 1)
    ## U1: binary unmeasured common cause of A and Y
    ## U2: binary unmeasured common cause of L and Y
    ## C: censoring variable
    ## Y: countinuous outcome
    
    ## specify dataframe
    df = pd.DataFrame()
    
    ## specify variables Z, U1, and U2
    Z_split = 0.4
    U1_split = 0.52
    U2_split = 0.23
    df['Z'] = np.random.choice([0, 1], size=n, replace=True, p=[Z_split, (1-Z_split)])
    df['U1'] = np.random.choice([0, 1], size=n, replace=True, p=[U1_split, (1-U1_split)])
    df['U2'] = np.random.choice([0, 1], size=n, replace=True, p=[U2_split, (1-U2_split)])
    
    ## specify variable A
    lambda_0 = -5.2
    lambda_1 = 9.3
    lambda_2 = 3.45   
    df['A'] = 0
    df.loc[(df['Z']==0) & (df['U1']==0), 'A'] = np.random.binomial(1, (1/(1+np.exp(-lambda_0))), size=df.loc[(df['Z']==0) & (df['U1']==0), 'A'].shape[0])
    df.loc[(df['Z']==1) & (df['U1']==0), 'A'] = np.random.binomial(1, (1/(1+np.exp(-(lambda_0+lambda_1)))), size=df.loc[(df['Z']==1) & (df['U1']==0), 'A'].shape[0])
    df.loc[(df['Z']==0) & (df['U1']==1), 'A'] = np.random.binomial(1, (1/(1+np.exp(-(lambda_0+lambda_2)))), size=df.loc[(df['Z']==0) & (df['U1']==1), 'A'].shape[0])
    df.loc[(df['Z']==1) & (df['U1']==1), 'A'] = np.random.binomial(1, (1/(1+np.exp(-(lambda_0+lambda_1+lambda_2)))), size=df.loc[(df['Z']==1) & (df['U1']==1), 'A'].shape[0])
    #model = smf.glm('A ~ Z + U1', data=df, family=sm.families.Binomial()).fit()
    #print(model.summary())
    #pd.crosstab(df['Z'], df['A'])
    #pd.crosstab(df['Z'], df['A'], normalize=True)
           
    ## specify variable L
    Beta_0 = -0.52
    Beta_1 = 2.32
    Beta_2 = 1.98
    df['L'] = 0
    df.loc[(df['A']==0) & (df['U2']==0), 'L'] = np.random.binomial(1, (1/(1+np.exp(-Beta_0))), size=df.loc[(df['A']==0) & (df['U2']==0), 'L'].shape[0])
    df.loc[(df['A']==1) & (df['U2']==0), 'L'] = np.random.binomial(1, (1/(1+np.exp(-(Beta_0+Beta_1)))), size=df.loc[(df['A']==1) & (df['U2']==0), 'L'].shape[0])
    df.loc[(df['A']==0) & (df['U2']==1), 'L'] = np.random.binomial(1, (1/(1+np.exp(-(Beta_0+Beta_2)))), size=df.loc[(df['A']==0) & (df['U2']==1), 'L'].shape[0])
    df.loc[(df['A']==1) & (df['U2']==1), 'L'] = np.random.binomial(1, (1/(1+np.exp(-(Beta_0+Beta_1+Beta_2)))), size=df.loc[(df['A']==1) & (df['U2']==1), 'L'].shape[0])
    #model = smf.glm('L ~ A + U2', data=df, family=sm.families.Binomial()).fit()
    #print(model.summary())
    
    ## specify variable C
    psi_0 = 0.15
    psi_1 = -5.8
    df['C'] = 0
    df.loc[df['L']==0, 'C'] = np.random.binomial(1, (1/(1+np.exp(-psi_0))), size=df.loc[df['L']==0, 'C'].shape[0])
    df.loc[df['L']==1, 'C'] = np.random.binomial(1, (1/(1+np.exp(-(psi_0+psi_1)))), size=df.loc[df['L']==1, 'C'].shape[0])
    #model = smf.glm('C ~ L', data=df, family=sm.families.Binomial()).fit()
    #print(model.summary())
    
    ## specify variable Y
    theta_0 = -0.5
    theta_1 = 5.78
    theta_2 = A_on_Y
    theta_3 = 1.42
    df['Y'] = theta_0 + (theta_1*df['U2']) + (theta_2*df['A']) + (theta_3*df['U1']) + np.random.normal(0, 0.5, df.shape[0])
    
    df_observed = df[['Z', 'A', 'L', 'C', 'Y']].copy()
    df_observed.loc[df['C']==1, 'Y'] = None
    
    return(df, df_observed)

In [3]:
#################################################################
## simulate dataset with no causal effect difference of A on Y ##
#################################################################
df, df_observed = simulate_dataset(n=2000000, A_on_Y=0)

In [4]:
#####################
## the "true" data ##
#####################
print(df.head(n=10))

   Z  U1  U2  A  L  C         Y
0  0   0   1  0  1  0  4.957074
1  1   0   1  1  1  0  5.717914
2  1   1   0  1  1  0  0.795860
3  0   1   1  0  0  1  6.636585
4  0   0   1  0  1  0  5.645637
5  0   1   1  0  1  0  7.528026
6  0   0   0  0  1  0 -0.839650
7  1   1   1  1  1  0  6.888819
8  1   0   0  1  1  0 -0.576333
9  1   0   0  1  0  1 -0.747018


In [5]:
##############################################
## data we get to "observe" in our analysis ##
##############################################
print(df_observed.head(n=10))

   Z  A  L  C         Y
0  0  0  1  0  4.957074
1  1  1  1  0  5.717914
2  1  1  1  0  0.795860
3  0  0  0  1       NaN
4  0  0  1  0  5.645637
5  0  0  1  0  7.528026
6  0  0  1  0 -0.839650
7  1  1  1  0  6.888819
8  1  1  1  0 -0.576333
9  1  1  0  1       NaN


In [6]:
#####################################################
## print percentage of censored observations (c=1) ##
#####################################################
print(df_observed['C'].value_counts(normalize=True))

0    0.922117
1    0.077883
Name: C, dtype: float64


In [7]:
def get_IPWs(df):
    
    PS_model = smf.glm('C ~ L', data=df, family=sm.families.Binomial()).fit()
    df['PS'] = PS_model.predict(df)
    df['IPW'] = 0
    df.loc[df['C']==1, 'IPW'] = 0
    df.loc[df['C']==0, 'IPW'] = 1 / (1 - df.loc[df['C']==0, 'PS'])
    
    return(df)

In [8]:
######################################################################
## get Inverse Probability Weights for Marginal Structural Modeling ##
######################################################################
df_observed = get_IPWs(df_observed)

In [9]:
#######################################################################
## ITT estimator of Z on Y (not adjusting for informative censoring) ##
#######################################################################
model = smf.ols(formula='Y ~ Z', data=df_observed).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     2874.
Date:                Sun, 20 Dec 2020   Prob (F-statistic):               0.00
Time:                        17:13:13   Log-Likelihood:            -4.2980e+06
No. Observations:             1844234   AIC:                         8.596e+06
Df Residuals:                 1844232   BIC:                         8.596e+06
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      4.9047      0.003   1627.123      0.0

In [10]:
###########################################################################
## ITT estimator of Z on Y, adjusting for informative censoring with MSM ##
###########################################################################
MSM_model = smf.wls('Y ~ Z', data=df_observed, weights=np.array(df_observed['IPW'], dtype=np.float64)).fit()
print(MSM_model.summary())

                            WLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.000
Model:                            WLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                   0.06952
Date:                Sun, 20 Dec 2020   Prob (F-statistic):              0.792
Time:                        17:13:28   Log-Likelihood:            -4.3891e+06
No. Observations:             1844234   AIC:                         8.778e+06
Df Residuals:                 1844232   BIC:                         8.778e+06
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      4.6310      0.003   1539.692      0.0

In [15]:
######################################################################
## simulate dataset with causal effect difference of A on Y of 0.12 ##
######################################################################
df, df_observed = simulate_dataset(n=2000000, A_on_Y=0.11)

In [16]:
######################################################################
## get Inverse Probability Weights for Marginal Structural Modeling ##
######################################################################
df_observed = get_IPWs(df_observed)

In [17]:
#############################
## ITT estimator of Z on Y ##
#############################
model = smf.ols(formula='Y ~ Z', data=df_observed).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     717.5
Date:                Sun, 20 Dec 2020   Prob (F-statistic):          5.14e-158
Time:                        17:29:23   Log-Likelihood:            -4.2974e+06
No. Observations:             1844024   AIC:                         8.595e+06
Df Residuals:                 1844022   BIC:                         8.595e+06
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      4.9165      0.003   1629.903      0.0

In [18]:
###########################################################################
## ITT estimator of Z on Y, adjusting for informative censoring with MSM ##
###########################################################################
MSM_model = smf.wls('Y ~ Z', data=df_observed, weights=np.array(df_observed['IPW'], dtype=np.float64)).fit()
print(MSM_model.summary())

                            WLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.000
Model:                            WLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     713.3
Date:                Sun, 20 Dec 2020   Prob (F-statistic):          4.05e-157
Time:                        17:29:28   Log-Likelihood:            -4.3887e+06
No. Observations:             1844024   AIC:                         8.777e+06
Df Residuals:                 1844022   BIC:                         8.778e+06
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      4.6413      0.003   1542.100      0.0